In [ ]:
import multiprocessing
import uuid
import os

if not os.path.exists('out'):
    os.makedirs('out')
if not os.path.exists('results'):
    os.makedirs('results')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.ticker import ScalarFormatter

from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs.latin_design import LatinDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction

In [ ]:
target_name = 'SEIZ_updated'
threads_per_run = multiprocessing.cpu_count()
init_exposures = 100
epsilon = 2
batch_size = 10

parameter_space = ParameterSpace([ContinuousParameter('p', 0, 1),
                                  ContinuousParameter('l', 0, 1),
                                  DiscreteParameter('prop_init_skeptics', np.arange(1, init_exposures, 1, dtype=np.int_))])

In [ ]:
def run_job(p, l, init_skeptics, init_adopters):
    with open(f'{target_name}_editable.fred', 'r') as file:
        filedata = file.read()

    vars = {
        'p': p, 
        'l': l, 
        'epsilon': epsilon, 
        'init_skeptic': int(init_skeptics), 
        'init_adopt': int(init_adopters)
    }

    run_uuid = uuid.uuid4()
    var_str = ', '.join(map(lambda item: f'{item[0]}={item[1]}', vars.items()))
    print(run_uuid, var_str)

    with open(f'out/{target_name}_{run_uuid}_out.fred', 'w') as file:
        file.write(filedata.format(**vars))
        
    !fred_job -p out/{target_name}_{run_uuid}_out.fred -k {target_name}_{run_uuid}_run -t {threads_per_run}
    !fred_csv -k {target_name}_{run_uuid}_run > results/{target_name}_{run_uuid}_run.csv

    with open(f'results/{target_name}_{run_uuid}_run.csv', 'r') as file:
        lines = file.readlines()

    with open(f'results/{target_name}_{run_uuid}_run.csv', 'w') as file:
        lines[0] = var_str+'\n'
        file.writelines(lines)

    return pd.read_csv(f'results/{target_name}_{run_uuid}_run.csv', header=2)

In [ ]:
def sample(X, clear_results=False):
    X = np.array(X)
    
    if X.ndim < 2:
        X = np.expand_dims(X, 0)
    elif X.ndim > 2:
        raise Exception(f'X has too many dimensions (ndim={X.ndim}, must be 1 or 2)')
    if X.shape[-1] != parameter_space.dimensionality:
        raise Exception(f'X has the wrong number of variables (variables={X.shape[-1]}, must be {parameter_space.dimensionality})')

    X = np.append(X, np.expand_dims(init_exposures-X[:,-1], axis=1), axis=1)  # create and add init_adopters variable

    if clear_results:
        !yes | fred_clear_all_results
    
    Y = []

    with multiprocessing.Pool() as pool:
        for data in pool.starmap(run_job, X):
            S_count = data['ADOPT.totS'].iat[0]
            Y.append([(-data['ADOPT.S'].diff()).max() / S_count])
        
    return np.vstack(Y)

In [ ]:
design = LatinDesign(parameter_space)
num_data_points = batch_size
X = design.get_samples(num_data_points)

In [ ]:
X

In [ ]:
Y = sample(X, clear_results=True)

In [ ]:
Y

In [ ]:
model_gpy = GPRegression(X,Y)
model_emukit = GPyModelWrapper(model_gpy)

In [ ]:
model_variance = ModelVariance(model=model_emukit)
expdesign_loop = ExperimentalDesignLoop(model=model_emukit,
                                        space=parameter_space,
                                        acquisition=model_variance,
                                        batch_size=batch_size)

max_iterations = 25
expdesign_loop.run_loop(sample, max_iterations)

In [ ]:
domain = []

for parameter in parameter_space.parameters:
    if isinstance(parameter, ContinuousParameter):
        domain.append(np.linspace(*parameter.bounds[0], 100))
    elif isinstance(parameter, DiscreteParameter):
        domain.append(parameter.domain)
    else:
        raise NotImplementedError(f'Domain computation not implemented for parameter type {type(parameter)}')

xx_p, yy_l, zz_init_skeptics = np.meshgrid(*domain)

In [ ]:
predictions = model_emukit.predict(np.stack((xx_p, yy_l, zz_init_skeptics), axis=-1).reshape(-1, parameter_space.dimensionality))[0].reshape(xx_p.shape)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_proj_type('ortho')

# init_slice = np.arange(0, zz_init_skeptics.shape[-1], zz_init_skeptics.shape[-1]/4, dtype=np.int_)
init_slice = 50

color_dim = predictions[:,:,init_slice]

ax.scatter(xx_p[:,:,init_slice], yy_l[:,:,init_slice], color_dim, c=color_dim, alpha=0.1)

ax.zaxis.set_major_formatter(ScalarFormatter(useOffset=False))

cmap = plt.get_cmap('viridis')
norm = plt.Normalize(color_dim.min(), color_dim.max())
sm = ScalarMappable(norm=norm, cmap=cmap)
cbar = fig.colorbar(sm, ax=ax)

ax.view_init(30, 30, 0)
plt.show()